# Blockchain.com

for bitcoin, get hashrate, transaction fees, nb transactions and average transactions per block

In [2]:
import pandas as pd
import requests
from datetime import datetime

In [3]:

def extract_data(response: requests, col_name1: str, col_name2: str) -> pd.DataFrame:
    """
    extract the data contained in response and return it into a dataframe with 2 columns (col_name1 and col_name2)
    """
    data = response.json()
    list_date = []
    list2 = []
    for point in data['values']:
        timestamp = point['x']
        value = point['y']
        date = pd.to_datetime(timestamp, unit='s').strftime('%Y-%m-%d')
        list_date.append(date)
        list2.append(value)

    # Création du DataFrame pandas
    df = pd.DataFrame({col_name1: list_date, col_name2: list2})
    return df

In [4]:
def add_block_reward(date: datetime) -> float:
    """ 
    return the block reward at the date 'date'
    """
    if date < datetime(2012, 11, 28):
        return 50
    elif date < datetime(2016, 7, 9):
        return 25
    elif date < datetime(2020, 5, 11):
        return 12.5
    else:
        return 6.25



In [5]:
start_date = '2010-01-01'
end_date = '2023-04-01'


In [6]:
#get the response for the hashrate----------------------------------------------------------------------------------------------------
#(TH/s)
response = requests.get(f'https://api.blockchain.info/charts/hash-rate?timespan=20year&start={start_date}&end={end_date}&format=json')
df_hashrate = extract_data(response, 'Date', 'Hashrate')

In [7]:
#get the response for the trans fees----------------------------------------------------------------------------------------------------
#(1 satoshi = 0,00000001 BTC)
response = requests.get(f'https://api.blockchain.info/charts/transaction-fees?timespan=20year&start={start_date}&end={end_date}&format=json')
df_fees = extract_data(response, 'Date', 'Fees(sat)')

In [8]:
#get the number of transactions----------------------------------------------------------------------------
response = requests.get(f'https://api.blockchain.info/charts/n-transactions?timespan=20year&start={start_date}&end={end_date}&format=json')
df_nb_trans = extract_data(response, 'Date', 'Nb_trans')


In [9]:
#get the number of transactions per bloc----------------------------------------------------------------------------
response = requests.get(f'https://api.blockchain.info/charts/n-transactions-per-block?timespan=20year&start={start_date}&end={end_date}&format=json')
df_nb_trans_block = extract_data(response, 'Date', 'Nb_trans_block')

In [10]:
response = requests.get(f'https://api.blockchain.info/charts/market-price?timespan=20year&start={start_date}&end={end_date}&format=json')
df_Price = extract_data(response, 'Date', 'Price')

In [11]:
#merge all those dataframes
merged_df = pd.merge(df_hashrate, df_fees, on='Date', how='outer')
merged_df = pd.merge(merged_df, df_nb_trans, on='Date', how='outer')
merged_df = pd.merge(merged_df, df_nb_trans_block, on='Date', how='outer')
merged_df = pd.merge(merged_df, df_Price, on='Date', how='outer')
#reward pour un bloc est block reward + Nb_trans_block * Fees
#pour 1 jour => 1 bloc toute les 10 minutes

print(merged_df)

            Date      Hashrate  Fees(sat)  Nb_trans  Nb_trans_block     Price
0     2010-01-01  6.997468e-06   0.000000     133.0        1.015267      0.00
1     2010-01-04  1.117243e-05   0.000000     183.0        1.011050      0.00
2     2010-01-07  9.349559e-06   0.000000     154.0        1.000000      0.00
3     2010-01-10  1.123123e-05   0.000000     190.0        1.000000      0.00
4     2010-01-13  1.025025e-05   0.000000     166.0        1.012195      0.00
...          ...           ...        ...       ...             ...       ...
1615  2023-04-08  3.499355e+08  21.895545  318505.0     2166.700680  27925.55
1616  2023-04-11  3.665991e+08  31.058914  385002.0     2500.012987  29656.24
1617  2023-04-14  3.332719e+08  27.739714  315959.0     2256.850000  30407.60
1618  2023-04-17  3.689796e+08  20.837154  316193.0     2039.954839  30315.39
1619  2023-04-20  3.193139e+08  21.239963  296683.0     2230.699248  28829.57

[1620 rows x 6 columns]


In [12]:
#add the fees for a block (Fees * Nb_trans_block) in bitcoin (*10^(-8))
computed_df = merged_df.copy()
computed_df['Fee_per_block'] = computed_df['Fees(sat)'] * computed_df['Nb_trans_block'] * 10**(-8)

# add the block reward
computed_df['Date'] = pd.to_datetime(computed_df['Date'])
computed_df['Block_reward'] = computed_df.apply(lambda row: add_block_reward(row['Date']), axis=1)

#estimate energy consumtion (for 1 year) at each point
#electricity price:0.05$/kwh
#1bloc/10mins
computed_df['ELectricity(TWh)'] = (6*24*365) * (computed_df['Block_reward'] + computed_df['Fee_per_block']) * computed_df['Price'] / (0.05)   * 10**(-9)

Halving history

Date : 28 novembre 2012
Récompense de bloc : 50 BTC -> 25 BTC

Date : 9 juillet 2016
Récompense de bloc : 25 BTC -> 12.5 BTC

Date : 11 mai 2020
Récompense de bloc : 12.5 BTC -> 6.25 BTC

(le block reward est initialement de 50 BTC, et que chaque fois que 210 000 blocs sont minés, le block reward est divisé par deux.)

# Cambridge and digiconomist

add the values from Cambridge and digiconomist

### Cambridge

In [13]:
df_cam = pd.read_csv('Cambridge_data.csv')

#delete unwanted columns
df_cam = df_cam.drop(columns=['Average electricity cost assumption: 0.05 USD/kWh', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4' ])

#rename the remaining columns
df_cam = df_cam.rename(columns={'Unnamed: 1':'Date', 'Unnamed: 5': 'Cambridge_MAX(TWh)', 'Unnamed: 6': 'Cambridge_MIN(TWh)', 'Unnamed: 7': 'Cambridge_GUESS(TWh)'})

#delete the first line, = name of the columns, could have been done in the read csv
df_cam = df_cam[df_cam['Date'] != 'Date and Time']

#convert dates
df_cam['Date'] = pd.to_datetime(df_cam['Date'])

Cambridge have data for everyday since 2010-07-18, left join on our dataframe to have on one day out of three

!!! Cambridge will have Nans between 2010-01-01 and 2010-07-18. Also Nans at the end if the new csv isn't dowloaded from their pages everytime (no API)

In [14]:
Cambridge_df = pd.merge(computed_df, df_cam, on='Date', how='left')

### Digiconomist

only since 2017

In [15]:
df_dig = pd.read_csv('bitcoin_digiconomist.csv')

df_dig = df_dig.rename(columns={'Unnamed: 0': 'Date', 'Unnamed: 1': 'Dig_Guess(TWh)', 'Unnamed: 2': 'Dig_Min(TWh)'})
df_dig = df_dig[df_dig['Date'] != 'Bitcoin Energy Consumption']
df_dig = df_dig[df_dig['Date'] != 'DateTime']
df_dig['Date'] = pd.to_datetime(df_dig['Date'])
df_dig

,Date,Dig_Guess(TWh),Dig_Min(TWh)
2,2017-02-10,"9,5856594507","3,3310264288"
3,2017-02-11,"9,5473353682","3,3162273608"
4,2017-02-12,"9,5787742594","3,1989691439"
5,2017-02-13,"9,4765061259","3,0451782162"
6,2017-02-14,"9,5448073164","3,1500624504"
...,...,...,...
2251,2023-04-09,"97,5718801163","97,5718801163"
2252,2023-04-10,"98,0690658421","98,0690658421"
2253,2023-04-11,"98,6682425809","98,6682425809"
2254,2023-04-12,"98,8758004848","98,8758004848"


In [16]:
Complete_df = pd.merge(Cambridge_df, df_dig, on='Date', how='left')

In [17]:
Complete_df.to_csv('../btc_values.csv', index=False)